The is an attempt using a deep learning network

https://keras.io/#you-have-just-found-keras

In [171]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras

In [153]:
import numpy as np
np.random.seed(1337)
import csv

In [154]:
# set model variables
batch_size = 128
nb_classes = 10
nb_epoch = 12

# number of convolutional filters to use
nb_filters = 32

# size of pooling area for max pooling
pool_size = (2, 2)

# convolution kernal size
kernel_size = (3, 3)

In [155]:
# input image dimensions
img_rows, img_cols = 28, 28

In [156]:
# read in data and reshape
dataset = np.genfromtxt(open('train.csv', 'r'), 
                        delimiter=",", skip_header=1)
print(dataset.shape)

(42000, 785)


In [157]:
y_train = dataset[:,0]
X_train = dataset[:,1:]
print(X_train.shape)
print(y_train.shape)

(42000, 784)
(42000,)


In [158]:
X_test = np.genfromtxt(open('test.csv', 'r'), 
                       delimiter =',', skip_header=1)
print(X_test.shape)

(28000, 784)


In [159]:
# reshape
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_train /= 255
print('X_train shape: ', X_train.shape)

X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test /= 255
print('X_test shape: ', X_test.shape)

('X_train shape: ', (42000, 28, 28, 1))
('X_test shape: ', (28000, 28, 28, 1))


In [160]:
input_shape = (img_rows, img_cols, 1)

In [161]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)

In [162]:
print(Y_train.shape)

(42000, 10)


In [163]:
# create cross validation set
cut_off = int(X_train.shape[0]*0.9)
print(cut_off)

37800


In [164]:
X_cv = X_train[cut_off:]
Y_cv = Y_train[cut_off:]

X_train = X_train[:cut_off]
Y_train = Y_train[:cut_off]

In [165]:
print(X_cv.shape)
print(Y_cv.shape)
print(X_train.shape)
print(Y_train.shape)

(4200, 28, 28, 1)
(4200, 10)
(37800, 28, 28, 1)
(37800, 10)


In [166]:
# model
model = Sequential()

In [167]:
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], border_mode='valid', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters*2, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters*2, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(144))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(144))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer='adadelta',
             metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_25 (Convolution2D) (None, 26, 26, 32)    320         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
activation_26 (Activation)       (None, 26, 26, 32)    0           convolution2d_25[0][0]           
____________________________________________________________________________________________________
convolution2d_26 (Convolution2D) (None, 24, 24, 32)    9248        activation_26[0][0]              
____________________________________________________________________________________________________
activation_27 (Activation)       (None, 24, 24, 32)    0           convolution2d_26[0][0]           
___________________________________________________________________________________________

In [177]:
# now fit model

model.fit(X_train, Y_train, batch_size=batch_size, 
          nb_epoch=nb_epoch, verbose=1, 
          validation_data=(X_cv, Y_cv),
          callbacks = [keras.callbacks.EarlyStopping()]
         )
score = model.evaluate(X_cv,Y_cv, verbose=0)
print('Test score: ', score[0])
print('Test accuracy: ', score[1])

Train on 37800 samples, validate on 4200 samples
Epoch 1/12
37800/37800 [==============================] - 180s - loss: 0.0473 - acc: 0.9870 - val_loss: 0.0261 - val_acc: 0.9926
Epoch 2/12
37800/37800 [==============================] - 178s - loss: 0.0429 - acc: 0.9885 - val_loss: 0.0273 - val_acc: 0.9914
('Test score: ', 0.027311888864472469)
('Test accuracy: ', 0.99142857142857144)


previous results

- ('Test score: ', 0.03304536611285238)
- ('Test accuracy: ', 0.98642857142857143)


- ('Test score: ', 0.026988923776163055)
- ('Test accuracy: ', 0.98999999999999999)


without early stops:
- ('Test score: ', 0.027871151176313148)
- ('Test accuracy: ', 0.99119047619047618)


with early stops:
- ('Test score: ', 0.027311888864472469)
- ('Test accuracy: ', 0.99142857142857144)

In [178]:
# now make predictions for the test set
predictions = model.predict_classes(X_test)

28000/28000 [==============================] - 51s    


In [179]:
labels = np.array(range(1, predictions.shape[0]+1))
results = np.column_stack([labels, predictions])

In [180]:
np.savetxt('results-v5-earlystop.csv', results, delimiter=',',
          header='ImageId,Label', fmt='%d', comments='')